# Driver Demand Forecasting
This notebook demonstrates a machine learning pipeline to forecast driver demand in ridesharing and food delivery services using time series analysis and ensemble models (XGBoost and CNN).

## Step 1: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

## Step 2: Load and Explore Sample Data

In [ ]:
np.random.seed(42)
weeks = np.tile(np.arange(1, 11), 10)
center_id = np.repeat(np.arange(1, 11), 10)
base_price = np.random.uniform(100, 200, size=100)
num_orders = (np.sin(weeks / 1.5) * 20 + np.random.normal(0, 5, size=100) + 50).astype(int)
data = pd.DataFrame({
    'week': weeks,
    'center_id': center_id,
    'base_price': base_price,
    'num_orders': num_orders
})
data.head()

## Step 3: Preprocess the Data

In [ ]:
X = data[['week', 'center_id', 'base_price']]
y = data['num_orders']
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Step 4: XGBoost Regressor Model

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost MAE:", mean_absolute_error(y_test, y_pred_xgb))
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))
print("XGBoost R²:", r2_score(y_test, y_pred_xgb))

## Step 5: CNN Regressor Model (1D)

In [ ]:
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

cnn_model = Sequential([
    Conv1D(32, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(1),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.fit(X_train_cnn, y_train, epochs=50, validation_split=0.2, callbacks=[EarlyStopping(patience=5)], verbose=0)

y_pred_cnn = cnn_model.predict(X_test_cnn)

print("CNN MAE:", mean_absolute_error(y_test, y_pred_cnn))
print("CNN RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_cnn)))
print("CNN R²:", r2_score(y_test, y_pred_cnn))

## Step 6: Compare Predictions

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(y_test.values, label='True')
plt.plot(y_pred_xgb, label='XGBoost')
plt.plot(y_pred_cnn, label='CNN')
plt.legend()
plt.title("Model Prediction Comparison")
plt.show()